In [ ]:
!nvidia-smi

In [ ]:
!pip install -q gdown ultralytics

In [ ]:
!pip install numpy==1.26.4 --force-reinstall --no-cache-dir

In [ ]:
import os
import gdown
from collections import defaultdict
import shutil
from pathlib import Path
import random
import random
from PIL import Image
import numpy as np
from itertools import combinations
from ultralytics import YOLO
from zipfile import ZipFile
from IPython.display import FileLink, display

In [ ]:
VEHICLES_DATASET_1_ID = "1IY6iL62T4M9BsgnPHT15MuHu83qis6e8"
PEOPLE_DATASET_ID = "1AxWGhDv4rnoH-uHv3757I_D4rLa-j6U6"
USE_FOLDERS = False

In [ ]:
def download_from_drive(file_id, output_path, is_folder=False):
    try:
        if is_folder:
            url = f"https://drive.google.com/drive/folders/{file_id}"
            gdown.download_folder(url, output=output_path, quiet=False, use_cookies=False)
        else:
            url = f"https://drive.google.com/uc?id={file_id}"
            gdown.download(url, output_path, quiet=False)
        return True
    except Exception as e:
        print(f"Error downloading: {e}")
        return False

print("Downloading datasets from Google Drive...")

if USE_FOLDERS:
    download_from_drive(VEHICLES_FOLDER_1_ID, "/kaggle/working/vehicles_dataset", is_folder=True)
    
    download_from_drive(PEOPLE_FOLDER_ID, "/kaggle/working/people_dataset", is_folder=True)
    
else:
    download_from_drive(VEHICLES_DATASET_1_ID, "/kaggle/working/vehicles_dataset_1.zip")
    
    download_from_drive(PEOPLE_DATASET_ID, "/kaggle/working/people_dataset.zip")
    
    !unzip -q /kaggle/working/vehicles_dataset_1.zip -d /kaggle/working/vehicles_dataset_1
    !unzip -q /kaggle/working/people_dataset.zip -d /kaggle/working/people_dataset
    
    print("Extraction complete!")

print("\nDatasets downloaded successfully!")

In [ ]:
def check_dataset_structure(base_path, dataset_name):
    print(f"{dataset_name}:")
    
    required_splits = ["train", "valid", "test"]
    structure_valid = True
    
    for split in required_splits:
        img_path = os.path.join(base_path, split, "images")
        lbl_path = os.path.join(base_path, split, "labels")
        
        img_exists = os.path.exists(img_path)
        lbl_exists = os.path.exists(lbl_path)
        
        img_count = len(os.listdir(img_path)) if img_exists else 0
        lbl_count = len(os.listdir(lbl_path)) if lbl_exists else 0
        
        status = "✅" if img_exists and lbl_exists else "❌"
        print(f"   {status} {split}: {img_count} images, {lbl_count} labels")
        
        if not (img_exists and lbl_exists):
            structure_valid = False
    
    print()
    return structure_valid

vehicles_valid_1 = check_dataset_structure("/kaggle/working/vehicles_dataset_1", "Vehicles Dataset_1")
vehicles_valid_2 = check_dataset_structure("/kaggle/input/cars-detection/Cars Detection", "Vehicles Dataset_2")
people_valid = check_dataset_structure("/kaggle/working/people_dataset", "People Dataset")

if vehicles_valid_1 and vehicles_valid_2 and people_valid:
    print("All datasets have correct structure!")
else:
    print("Warning: Some datasets may have incorrect structure")

In [ ]:
COMBINED_DATASET = "/kaggle/working/combined_dataset"

VEHICLE_1_REMAP = {
    0: 0,
    1: 0,
    2: 0,
    3: 0,
    4: 0,
    5: 0,
    6: 0,
    7: 0,
    8: 0,
    9: 0,
    10: 0,
    11: 0
}

VEHICLE_2_REMAP ={
    0: 0,
    1: 0,
    2: 0,
    3: 1,
    4: 0
}

CLASS_NAMES = ['car', 'bike']

In [ ]:
def create_directory_structure(base_path):
    print(f"Creating directory structure at {base_path}")
    
    for sub in ["images/train", "images/valid", "images/test",
                "labels/train", "labels/valid", "labels/test"]:
        os.makedirs(os.path.join(base_path, sub), exist_ok=True)
    
    print("Directory structure created")

In [ ]:
def copy_dataset(src_base, dst_base, class_mapping=None, dataset_name="Dataset", prefix=""):
    print(f"\nProcessing {dataset_name}...")
    
    stats = {"train": 0, "valid": 0, "test": 0}
    skipped_stats = defaultdict(lambda: defaultdict(int))
    unmapped_classes = set()
    
    for split in ["train", "valid", "test"]:
        src_images = os.path.join(src_base, split, "images")
        dst_images = os.path.join(dst_base, "images", split)
        
        if os.path.exists(src_images):
            image_files = [f for f in os.listdir(src_images) 
                          if f.lower().endswith(('.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG'))]
            
            for f in image_files:
                src_path = os.path.join(src_images, f)
                new_filename = f"{prefix}{f}"
                dst_path = os.path.join(dst_images, new_filename)
                shutil.copy(src_path, dst_path)
                stats[split] += 1
        
        src_labels = os.path.join(src_base, split, "labels")
        dst_labels = os.path.join(dst_base, "labels", split)
        
        if os.path.exists(src_labels):
            for f in os.listdir(src_labels):
                if not f.endswith('.txt'):
                    continue
                    
                src_file = os.path.join(src_labels, f)
                new_filename = f"{prefix}{f}"
                dst_file = os.path.join(dst_labels, new_filename)
                
                with open(src_file, "r") as fr:
                    lines = fr.readlines()
                
                new_lines = []
                
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) < 5:
                        skipped_stats[split]["invalid_format"] += 1
                        continue
                    
                    try:
                        class_id = int(parts[0])
                    except ValueError:
                        skipped_stats[split]["invalid_class_id"] += 1
                        continue
                    
                    if class_mapping is not None:
                        if class_id in class_mapping:
                            parts[0] = str(class_mapping[class_id])
                            new_lines.append(" ".join(parts))
                        else:
                            unmapped_classes.add(class_id)
                            skipped_stats[split]["unmapped_class"] += 1
                    else:
                        new_lines.append(" ".join(parts))
                
                if new_lines:
                    with open(dst_file, "w") as fw:
                        fw.write("\n".join(new_lines) + "\n")
                else:
                    skipped_stats[split]["empty_after_filter"] += 1
    
    print(f"Train: {stats['train']} images")
    print(f"Valid: {stats['valid']} images")
    print(f"Test: {stats['test']} images")
    
    total_skipped = sum(sum(split_stats.values()) for split_stats in skipped_stats.values())
    if total_skipped > 0:
        print(f"\nSkipped annotations:")
        for split, reasons in skipped_stats.items():
            if reasons:
                print(f"      {split}:")
                for reason, count in reasons.items():
                    print(f"         - {reason}: {count}")
    
    if unmapped_classes:
        print(f"\nFound unmapped class IDs: {sorted(unmapped_classes)}")
            
    return stats, skipped_stats, unmapped_classes

In [ ]:
def create_data_yaml(base_path, class_names):
    data_yaml = f"""# Dataset configuration for YOLO training
path: {base_path}
train: images/train
val: images/valid
test: images/test

# Number of classes
nc: {len(class_names)}

# Class names
names: {class_names}
"""
    
    yaml_path = os.path.join(base_path, "data.yaml")
    with open(yaml_path, "w") as f:
        f.write(data_yaml)
    
    print(f"\ndata.yaml created at {yaml_path}")
    return yaml_path

In [ ]:
def verify_combined_dataset(base_path):
    print("\nVerifying combined dataset...")
    
    total_images = 0
    total_labels = 0
    
    for split in ["train", "valid", "test"]:
        img_dir = os.path.join(base_path, "images", split)
        lbl_dir = os.path.join(base_path, "labels", split)
        
        img_count = len([f for f in os.listdir(img_dir) 
                        if f.lower().endswith(('.jpg', '.jpeg', '.png'))]) if os.path.exists(img_dir) else 0
        lbl_count = len([f for f in os.listdir(lbl_dir) 
                        if f.endswith('.txt')]) if os.path.exists(lbl_dir) else 0
        
        total_images += img_count
        total_labels += lbl_count
        
        print(f"   {split.capitalize()}: {img_count} images, {lbl_count} labels")
        
        if img_count != lbl_count:
            print(f"Warning: Mismatch in {split} set!")
    
    print(f"\nTotal: {total_images} images, {total_labels} labels")

In [ ]:
def analyze_dataset_classes(dataset_path, dataset_name):
    print(f"\nAnalyzing {dataset_name}...")
    
    class_counts = {}
    total_annotations = 0
    
    for split in ["train", "valid", "test"]:
        labels_dir = os.path.join(dataset_path, split, "labels")
        
        if not os.path.exists(labels_dir):
            continue
            
        for label_file in os.listdir(labels_dir):
            if not label_file.endswith('.txt'):
                continue
                
            with open(os.path.join(labels_dir, label_file), 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        try:
                            class_id = int(parts[0])
                            class_counts[class_id] = class_counts.get(class_id, 0) + 1
                            total_annotations += 1
                        except ValueError:
                            continue
    
    print(f"Total annotations: {total_annotations}")
    print(f"Class distribution:")
    for class_id in sorted(class_counts.keys()):
        count = class_counts[class_id]
        percentage = (count / total_annotations * 100) if total_annotations > 0 else 0
        print(f"      Class {class_id}: {count:,} annotations ({percentage:.1f}%)")
    
    return class_counts

print("=" * 70)
print("DATASET CLASS ANALYSIS")
print("=" * 70)

vehicle_classes_1 = analyze_dataset_classes("/kaggle/working/vehicles_dataset_1", "Vehicles 1 Dataset")
vehicle_classes_2 = analyze_dataset_classes("/kaggle/input/cars-detection/Cars Detection", "Vehicles 2 Dataset")
people_classes = analyze_dataset_classes("/kaggle/working/people_dataset", "People Dataset")

print("\n" + "=" * 70)
print("RECOMMENDATIONS")
print("=" * 70)

if people_classes:
    people_class_ids = set(people_classes.keys())
    if people_class_ids != {0}:
        print(f"\nPeople dataset has classes: {sorted(people_class_ids)}")
        print(f"   Your person_mapping only maps class 0")
        print(f"   You need to map ALL these classes to 0 (person)")
        print(f"\n   Suggested fix:")
        print(f"   person_mapping = {{")
        for class_id in sorted(people_class_ids):
            print(f"       {class_id}: 0,  # person")
        print(f"   }}")
    else:
        print("People dataset looks correct (only class 0)")

if vehicle_classes_1:
    vehicle_class_ids = set(vehicle_classes_1.keys())
    unmapped = vehicle_class_ids - set(VEHICLE_1_REMAP.keys())
    if unmapped:
        print(f"\nVehicle dataset has unmapped classes: {sorted(unmapped)}")
        print(f"   Add these to VEHICLE_REMAP")

if vehicle_classes_2:
    vehicle_class_ids = set(vehicle_classes_2.keys())
    unmapped = vehicle_class_ids - set(VEHICLE_2_REMAP.keys())
    if unmapped:
        print(f"\nVehicle dataset has unmapped classes: {sorted(unmapped)}")
        print(f"   Add these to VEHICLE_REMAP")

In [ ]:
if os.path.exists(COMBINED_DATASET):
    shutil.rmtree(COMBINED_DATASET)
    print("Removed old dataset\n")

create_directory_structure(COMBINED_DATASET)

print("\n" + "="*50)
veh_stats, veh_skipped, veh_unmapped = copy_dataset(
    src_base="/kaggle/working/vehicles_dataset_1",
    dst_base=COMBINED_DATASET,
    class_mapping=VEHICLE_1_REMAP,
    dataset_name="Vehicle Dataset",
    prefix="veh1_"
)

print("\n" + "="*50)
veh_stats, veh_skipped, veh_unmapped = copy_dataset(
    src_base="/kaggle/input/cars-detection/Cars Detection",
    dst_base=COMBINED_DATASET,
    class_mapping=VEHICLE_2_REMAP,
    dataset_name="Vehicle Dataset",
    prefix="veh2_"
)

yaml_path = create_data_yaml(COMBINED_DATASET, CLASS_NAMES)

print("\n" + "="*70)
print("BEFORE CLEANUP")
print("="*70)
verify_combined_dataset(COMBINED_DATASET)


def clean_unmatched_images(dataset_path):
    print("\nCleaning up images without labels...")
    
    total_deleted = 0
    
    for split in ["train", "valid", "test"]:
        img_dir = os.path.join(dataset_path, "images", split)
        lbl_dir = os.path.join(dataset_path, "labels", split)
        
        if not os.path.exists(img_dir):
            continue
        
        deleted_count = 0
        image_files = [f for f in os.listdir(img_dir) 
                      if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        for img_file in image_files:
            img_name = os.path.splitext(img_file)[0]
            label_file = f"{img_name}.txt"
            label_path = os.path.join(lbl_dir, label_file)
            
            if not os.path.exists(label_path):
                img_path = os.path.join(img_dir, img_file)
                os.remove(img_path)
                deleted_count += 1
                total_deleted += 1
        
        if deleted_count > 0:
            print(f"   Deleted {deleted_count} images without labels in {split}")
    
    print(f"\nTotal images deleted: {total_deleted}")
    return total_deleted

print("\n" + "="*70)
deleted = clean_unmatched_images(COMBINED_DATASET)

print("\n" + "="*70)
print("AFTER CLEANUP")
print("="*70)
verify_combined_dataset(COMBINED_DATASET)

print("\n" + "=" * 70)
print("DATASET PREPARATION COMPLETE!")
print("=" * 70)
print(f"\nCombined Dataset: {COMBINED_DATASET}")
print(f"Config File: {yaml_path}")
print(f"Classes: {CLASS_NAMES}")
print(f"Removed: {deleted} images without labels")

In [ ]:
def balance_dataset(dataset_path, target_ratio=1.0):    
    for split in ["train", "valid", "test"]:
        img_dir = os.path.join(dataset_path, "images", split)
        lbl_dir = os.path.join(dataset_path, "labels", split)
        
        if not os.path.exists(img_dir) or not os.path.exists(lbl_dir):
            print(f"\n{split}: Directory not found, skipping...")
            continue
        
        class_0_files = []  # Images containing class 0 (people)
        class_1_files = []  # Images containing class 1 (cars)
        both_classes_files = []  # Images containing both classes
        no_label_files = []  # Images with no labels or other classes
        
        for img_file in os.listdir(img_dir):
            if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue
                
            lbl_file = os.path.splitext(img_file)[0] + '.txt'
            lbl_path = os.path.join(lbl_dir, lbl_file)
            
            if not os.path.exists(lbl_path):
                no_label_files.append(img_file)
                continue
            
            has_class_0 = False
            has_class_1 = False
            
            try:
                with open(lbl_path, 'r') as f:
                    for line in f:
                        parts = line.strip().split()
                        if len(parts) > 0:
                            class_id = int(parts[0])
                            if class_id == 0:
                                has_class_0 = True
                            elif class_id == 1:
                                has_class_1 = True
            except Exception as e:
                print(f"Error reading {lbl_file}: {e}")
                continue
            
            if has_class_0 and has_class_1:
                both_classes_files.append(img_file)
            elif has_class_0:
                class_0_files.append(img_file)
            elif has_class_1:
                class_1_files.append(img_file)
            else:
                no_label_files.append(img_file)
        
        print(f"\n{split}:")
        print(f"Class 0 only (people): {len(class_0_files)} images")
        print(f"Class 1 only (cars): {len(class_1_files)} images")
        print(f"Both classes: {len(both_classes_files)} images")
        print(f"No relevant labels: {len(no_label_files)} images")
        
        total_class_0 = len(class_0_files) + len(both_classes_files)
        total_class_1 = len(class_1_files) + len(both_classes_files)
        
        print(f"Total with class 0: {total_class_0}")
        print(f"Total with class 1: {total_class_1}")
        
        if len(class_0_files) > len(class_1_files):
            target_count = int(len(class_1_files) * target_ratio)
            files_to_remove = balance_class(class_0_files, target_count, img_dir, lbl_dir)
            print(f"      Undersampling class 0 (people)")
            print(f"      Removed: {files_to_remove} images")
            print(f"      New class 0 only count: {len(class_0_files) - files_to_remove}")
            
        elif len(class_1_files) > len(class_0_files):
            target_count = int(len(class_0_files) * target_ratio)
            files_to_remove = balance_class(class_1_files, target_count, img_dir, lbl_dir)
            print(f"Undersampling class 1 (cars)")
            print(f"Removed: {files_to_remove} images")
            print(f"New class 1 only count: {len(class_1_files) - files_to_remove}")
        else:
            print(f"Already balanced")
            continue
        
        remaining_class_0_only = len([f for f in os.listdir(img_dir) 
                                      if f in class_0_files and os.path.exists(os.path.join(img_dir, f))])
        remaining_class_1_only = len([f for f in os.listdir(img_dir) 
                                      if f in class_1_files and os.path.exists(os.path.join(img_dir, f))])
        
        ratio = remaining_class_0_only / remaining_class_1_only if remaining_class_1_only > 0 else 0
        print(f"Final ratio (class 0 only : class 1 only) = {ratio:.2f}:1")
        print(f"Images with both classes retained: {len(both_classes_files)}")


def balance_class(file_list, target_count, img_dir, lbl_dir):
    if len(file_list) <= target_count:
        return 0
    
    random.seed(42)
    files_to_keep = set(random.sample(file_list, target_count))
    files_to_remove_list = [f for f in file_list if f not in files_to_keep]
    
    removed_count = 0
    for img_file in files_to_remove_list:
        img_path = os.path.join(img_dir, img_file)
        if os.path.exists(img_path):
            os.remove(img_path)
        
        lbl_file = os.path.splitext(img_file)[0] + '.txt'
        lbl_path = os.path.join(lbl_dir, lbl_file)
        if os.path.exists(lbl_path):
            os.remove(lbl_path)
        
        removed_count += 1
    
    return removed_count


def verify_balanced_dataset(dataset_path):
    for split in ["train", "valid", "test"]:
        img_dir = os.path.join(dataset_path, "images", split)
        lbl_dir = os.path.join(dataset_path, "labels", split)
        
        if not os.path.exists(img_dir) or not os.path.exists(lbl_dir):
            continue
        
        class_0_count = 0
        class_1_count = 0
        both_count = 0
        total_images = 0
        
        for img_file in os.listdir(img_dir):
            if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue
            
            total_images += 1
            lbl_file = os.path.splitext(img_file)[0] + '.txt'
            lbl_path = os.path.join(lbl_dir, lbl_file)
            
            if not os.path.exists(lbl_path):
                continue
            
            has_class_0 = False
            has_class_1 = False
            
            with open(lbl_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) > 0:
                        class_id = int(parts[0])
                        if class_id == 0:
                            has_class_0 = True
                        elif class_id == 1:
                            has_class_1 = True
            
            if has_class_0 and has_class_1:
                both_count += 1
            elif has_class_0:
                class_0_count += 1
            elif has_class_1:
                class_1_count += 1
        
        print(f"\n{split.upper()}:")
        print(f"   Total images: {total_images}")
        print(f"   Class 0 only: {class_0_count}")
        print(f"   Class 1 only: {class_1_count}")
        print(f"   Both classes: {both_count}")
        print(f"   Ratio (0:1): {class_0_count/class_1_count if class_1_count > 0 else 'N/A':.2f}")


COMBINED_DATASET = "/kaggle/working/combined_dataset"
balance_dataset(COMBINED_DATASET, target_ratio=1.0)
verify_balanced_dataset(COMBINED_DATASET)

In [ ]:
def create_mixed_class_images(dataset_path, output_path, num_mixed=500, split_ratio=(0.7, 0.2, 0.1)):
    random.seed(42)
    
    for split in ["train", "valid", "test"]:
        img_dir = os.path.join(dataset_path, "images", split)
        lbl_dir = os.path.join(dataset_path, "labels", split)
        
        out_img_dir = os.path.join(output_path, "images", split)
        out_lbl_dir = os.path.join(output_path, "labels", split)
        os.makedirs(out_img_dir, exist_ok=True)
        os.makedirs(out_lbl_dir, exist_ok=True)
        
        if not os.path.exists(img_dir) or not os.path.exists(lbl_dir):
            continue
        
        class_images = {}
        
        for img_file in os.listdir(img_dir):
            if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue
            
            lbl_file = os.path.splitext(img_file)[0] + '.txt'
            lbl_path = os.path.join(lbl_dir, lbl_file)
            
            if not os.path.exists(lbl_path):
                continue
            
            with open(lbl_path, 'r') as f:
                classes = set()
                for line in f:
                    parts = line.strip().split()
                    if len(parts) > 0:
                        classes.add(int(parts[0]))
                
                if len(classes) == 1:
                    cls = list(classes)[0]
                    if cls not in class_images:
                        class_images[cls] = []
                    class_images[cls].append(img_file)
        
        print(f"\n{split}:")
        for cls, images in sorted(class_images.items()):
            print(f"Class {cls} images available: {len(images)}")
        
        if len(class_images) < 2:
            print(f"Not enough classes to create mixed samples")
            continue
        
        class_ids = sorted(class_images.keys())
        all_combinations = []
        
        for combo in combinations(class_ids, 2):
            all_combinations.append(combo)
        
        if len(class_ids) >= 3:
            for combo in combinations(class_ids, 3):
                all_combinations.append(combo)
        
        print(f"Creating combinations: {all_combinations}")
        
        if split == "train":
            num_to_create = int(num_mixed * split_ratio[0])
        elif split == "valid":
            num_to_create = int(num_mixed * split_ratio[1])
        else:
            num_to_create = int(num_mixed * split_ratio[2])
        
        images_per_combo = max(1, num_to_create // len(all_combinations))
        
        created = 0
        mix_idx = 0
        
        for combo in all_combinations:
            combo_created = 0
            
            for i in range(images_per_combo):
                try:
                    selected_images = []
                    selected_labels = []
                    
                    for cls in combo:
                        if len(class_images[cls]) == 0:
                            continue
                        img_file = random.choice(class_images[cls])
                        selected_images.append(os.path.join(img_dir, img_file))
                        selected_labels.append(os.path.join(lbl_dir, os.path.splitext(img_file)[0] + '.txt'))
                    
                    if len(selected_images) < 2:
                        continue
                    
                    if len(selected_images) == 2:
                        mixed_img, mixed_labels = combine_two_images(
                            selected_images[0], selected_images[1],
                            selected_labels[0], selected_labels[1]
                        )
                    else:
                        mixed_img, mixed_labels = combine_multiple_images(
                            selected_images, selected_labels
                        )
                    
                    mixed_name = f"mixed_{split}_{mix_idx:05d}.jpg"
                    mixed_img.save(os.path.join(out_img_dir, mixed_name))
                    
                    with open(os.path.join(out_lbl_dir, f"mixed_{split}_{mix_idx:05d}.txt"), 'w') as f:
                        f.write(mixed_labels)
                    
                    created += 1
                    combo_created += 1
                    mix_idx += 1
                    
                except Exception as e:
                    print(f"Error creating mixed image: {e}")
            
            print(f"Created {combo_created} images for combination {combo}")
        
        print(f"Total created: {created} mixed-class images")


def combine_two_images(img0_path, img1_path, lbl0_path, lbl1_path):
    img0 = Image.open(img0_path).convert('RGB')
    img1 = Image.open(img1_path).convert('RGB')
    
    target_height = 640
    img0 = img0.resize((int(img0.width * target_height / img0.height), target_height))
    img1 = img1.resize((int(img1.width * target_height / img1.height), target_height))
    
    total_width = img0.width + img1.width
    combined = Image.new('RGB', (total_width, target_height))
    combined.paste(img0, (0, 0))
    combined.paste(img1, (img0.width, 0))
    
    combined_labels = []
    
    with open(lbl0_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 5:
                cls, x_center, y_center, width, height = map(float, parts[:5])
                new_x_center = (x_center * img0.width) / total_width
                new_width = (width * img0.width) / total_width
                combined_labels.append(f"{int(cls)} {new_x_center:.6f} {y_center:.6f} {new_width:.6f} {height:.6f}")
    
    with open(lbl1_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 5:
                cls, x_center, y_center, width, height = map(float, parts[:5])
                new_x_center = ((x_center * img1.width) + img0.width) / total_width
                new_width = (width * img1.width) / total_width
                combined_labels.append(f"{int(cls)} {new_x_center:.6f} {y_center:.6f} {new_width:.6f} {height:.6f}")
    
    return combined, '\n'.join(combined_labels)


def combine_multiple_images(img_paths, lbl_paths):
    images = [Image.open(p).convert('RGB') for p in img_paths]
    
    target_height = 640
    
    resized = []
    for img in images:
        new_width = int(img.width * target_height / img.height)
        resized.append(img.resize((new_width, target_height)))
    
    if len(resized) == 3:
        top_width = resized[0].width + resized[1].width
        bottom_width = resized[2].width
        total_width = max(top_width, bottom_width)
        total_height = target_height * 2
        
        combined = Image.new('RGB', (total_width, total_height))
        combined.paste(resized[0], (0, 0))
        combined.paste(resized[1], (resized[0].width, 0))
        combined.paste(resized[2], ((total_width - resized[2].width) // 2, target_height))
        
        combined_labels = []
        
        with open(lbl_paths[0], 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    cls, x_center, y_center, width, height = map(float, parts[:5])
                    new_x_center = (x_center * resized[0].width) / total_width
                    new_y_center = (y_center * target_height) / total_height
                    new_width = (width * resized[0].width) / total_width
                    new_height = (height * target_height) / total_height
                    combined_labels.append(f"{int(cls)} {new_x_center:.6f} {new_y_center:.6f} {new_width:.6f} {new_height:.6f}")
        
        with open(lbl_paths[1], 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    cls, x_center, y_center, width, height = map(float, parts[:5])
                    new_x_center = ((x_center * resized[1].width) + resized[0].width) / total_width
                    new_y_center = (y_center * target_height) / total_height
                    new_width = (width * resized[1].width) / total_width
                    new_height = (height * target_height) / total_height
                    combined_labels.append(f"{int(cls)} {new_x_center:.6f} {new_y_center:.6f} {new_width:.6f} {new_height:.6f}")
        
        x_offset = (total_width - resized[2].width) // 2
        with open(lbl_paths[2], 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    cls, x_center, y_center, width, height = map(float, parts[:5])
                    new_x_center = ((x_center * resized[2].width) + x_offset) / total_width
                    new_y_center = ((y_center * target_height) + target_height) / total_height
                    new_width = (width * resized[2].width) / total_width
                    new_height = (height * target_height) / total_height
                    combined_labels.append(f"{int(cls)} {new_x_center:.6f} {new_y_center:.6f} {new_width:.6f} {new_height:.6f}")
        
        return combined, '\n'.join(combined_labels)
    
    else:
        return combine_two_images(img_paths[0], img_paths[1], lbl_paths[0], lbl_paths[1])


def merge_datasets(original_path, mixed_path, output_path):
    for split in ["train", "valid", "test"]:
        out_img_dir = os.path.join(output_path, "images", split)
        out_lbl_dir = os.path.join(output_path, "labels", split)
        os.makedirs(out_img_dir, exist_ok=True)
        os.makedirs(out_lbl_dir, exist_ok=True)
        
        orig_img_dir = os.path.join(original_path, "images", split)
        orig_lbl_dir = os.path.join(original_path, "labels", split)
        
        if os.path.exists(orig_img_dir):
            for img_file in os.listdir(orig_img_dir):
                shutil.copy2(
                    os.path.join(orig_img_dir, img_file),
                    os.path.join(out_img_dir, img_file)
                )
        
        if os.path.exists(orig_lbl_dir):
            for lbl_file in os.listdir(orig_lbl_dir):
                shutil.copy2(
                    os.path.join(orig_lbl_dir, lbl_file),
                    os.path.join(out_lbl_dir, lbl_file)
                )
        
        # Copy mixed images
        mixed_img_dir = os.path.join(mixed_path, "images", split)
        mixed_lbl_dir = os.path.join(mixed_path, "labels", split)
        
        if os.path.exists(mixed_img_dir):
            for img_file in os.listdir(mixed_img_dir):
                shutil.copy2(
                    os.path.join(mixed_img_dir, img_file),
                    os.path.join(out_img_dir, img_file)
                )
        
        if os.path.exists(mixed_lbl_dir):
            for lbl_file in os.listdir(mixed_lbl_dir):
                shutil.copy2(
                    os.path.join(mixed_lbl_dir, lbl_file),
                    os.path.join(out_lbl_dir, lbl_file)
                )
        
        total_images = len(os.listdir(out_img_dir))
        print(f"   {split}: {total_images} total images")


ORIGINAL_DATASET = "/kaggle/working/combined_dataset"
MIXED_OUTPUT = "/kaggle/working/mixed_images_temp"
FINAL_DATASET = "/kaggle/working/final_balanced_dataset"

create_mixed_class_images(
    dataset_path=ORIGINAL_DATASET,
    output_path=MIXED_OUTPUT,
    num_mixed= 500,
    split_ratio=(0.7, 0.2, 0.1)
)

merge_datasets(ORIGINAL_DATASET, MIXED_OUTPUT, FINAL_DATASET)

print("\nDataset creation complete!")
print(f"New dataset location: {FINAL_DATASET}")

In [ ]:
yaml_path_fin = create_data_yaml("/kaggle/working/final_balanced_dataset", CLASS_NAMES)

In [ ]:
def create_people_dataset(source_dataset, output_dataset):
    for split in ["train", "valid", "test"]:
        src_img_dir = os.path.join(source_dataset, "images", split)
        src_lbl_dir = os.path.join(source_dataset, "labels", split)
        
        dst_img_dir = os.path.join(output_dataset, "images", split)
        dst_lbl_dir = os.path.join(output_dataset, "labels", split)
        os.makedirs(dst_img_dir, exist_ok=True)
        os.makedirs(dst_lbl_dir, exist_ok=True)
        
        if not os.path.exists(src_img_dir):
            continue
        
        copied = 0
        for img_file in os.listdir(src_img_dir):
            if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue
            
            lbl_file = os.path.splitext(img_file)[0] + '.txt'
            src_lbl_path = os.path.join(src_lbl_dir, lbl_file)
            
            if not os.path.exists(src_lbl_path):
                continue
            
            has_people = False
            filtered_labels = []
            
            with open(src_lbl_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        cls = int(parts[0])
                        if cls == 0:  # People class
                            has_people = True
                            filtered_labels.append(f"0 {' '.join(parts[1:])}\n")
            
            if has_people:
                shutil.copy2(
                    os.path.join(src_img_dir, img_file),
                    os.path.join(dst_img_dir, img_file)
                )
                
                with open(os.path.join(dst_lbl_dir, lbl_file), 'w') as f:
                    f.writelines(filtered_labels)
                
                copied += 1
        
        print(f"   {split}: {copied} images with people")
    
    yaml_content = f"""path: {output_dataset}
train: images/train
val: images/valid
test: images/test

nc: 1
names: ['person']
"""
    
    with open(os.path.join(output_dataset, "data.yaml"), 'w') as f:
        f.write(yaml_content)
    
    print(f"People dataset created at: {output_dataset}")


def create_vehicle_dataset(source_dataset, output_dataset, vehicle_classes=[1, 2]):
    print("Creating vehicle-only dataset...")
    
    class_mapping = {vehicle_classes[0]: 0, vehicle_classes[1]: 1}
    
    for split in ["train", "valid", "test"]:
        src_img_dir = os.path.join(source_dataset, "images", split)
        src_lbl_dir = os.path.join(source_dataset, "labels", split)
        
        dst_img_dir = os.path.join(output_dataset, "images", split)
        dst_lbl_dir = os.path.join(output_dataset, "labels", split)
        os.makedirs(dst_img_dir, exist_ok=True)
        os.makedirs(dst_lbl_dir, exist_ok=True)
        
        if not os.path.exists(src_img_dir):
            continue
        
        copied = 0
        for img_file in os.listdir(src_img_dir):
            if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue
            
            lbl_file = os.path.splitext(img_file)[0] + '.txt'
            src_lbl_path = os.path.join(src_lbl_dir, lbl_file)
            
            if not os.path.exists(src_lbl_path):
                continue
            
            has_vehicle = False
            filtered_labels = []
            
            with open(src_lbl_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        cls = int(parts[0])
                        if cls in class_mapping:  # Vehicle class
                            has_vehicle = True
                            new_cls = class_mapping[cls]
                            filtered_labels.append(f"{new_cls} {' '.join(parts[1:])}\n")
            
            if has_vehicle:
                shutil.copy2(
                    os.path.join(src_img_dir, img_file),
                    os.path.join(dst_img_dir, img_file)
                )
                
                with open(os.path.join(dst_lbl_dir, lbl_file), 'w') as f:
                    f.writelines(filtered_labels)
                
                copied += 1
        
        print(f"   {split}: {copied} images with vehicles")
    
    yaml_content = f"""path: {output_dataset}
train: images/train
val: images/valid
test: images/test

nc: 2
names: ['bike', 'car']
"""
    
    with open(os.path.join(output_dataset, "data.yaml"), 'w') as f:
        f.write(yaml_content)
    
    print(f"Vehicle dataset created at: {output_dataset}")


SOURCE_DATASET = "/kaggle/working/combined_dataset"
PEOPLE_DATASET = "/kaggle/working/people_dataset"
VEHICLE_DATASET = "/kaggle/working/vehicle_dataset"

print("="*70)
print("🔀 SPLITTING DATASET FOR DUAL MODEL TRAINING")
print("="*70)

create_people_dataset(SOURCE_DATASET, PEOPLE_DATASET)

print()
create_vehicle_dataset(SOURCE_DATASET, VEHICLE_DATASET, vehicle_classes=[1, 2])

In [ ]:
print("Training people detector...")
people_model = YOLO('yolo11s.pt')
people_model.train(
    data='/kaggle/working/people_dataset/data.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    name='people_detector',
    patience=20
)

In [ ]:
vehicle_model = YOLO('yolo11s.pt')
vehicle_model.train(
    data='/kaggle/working/final_balanced_dataset/data.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    name='vehicle_detector',
    patience=5
)

In [ ]:
folders_to_zip = [
    "/kaggle/working/runs/detect/vehicle_detector"
]

zip_filename = "/kaggle/working/car.zip"

with ZipFile(zip_filename, 'w') as zipf:
    for folder in folders_to_zip:
        folder_path = Path(folder)
        for file in folder_path.rglob("*"):
            zipf.write(file, arcname=file.relative_to(folder_path.parent))

print(f"Zip file created: {zip_filename}")

display(FileLink(zip_filename))